# Task 2

In [36]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

train_in = pd.read_csv('train_in.csv', header=None)
train_out = pd.read_csv('train_out.csv', header=None)
test_in = pd.read_csv('test_in.csv', header=None)
test_out = pd.read_csv('test_out.csv', header=None)

In [19]:
# insert a column of one in the train set for the bias
train_in.insert(0, 'w0', 1)
test_in.insert(0, 'w0', 1)

# generate Weights randomly
rng = default_rng(seed=42) # ?-> do i have to change the seed different and calculate the mean
vals = rng.standard_normal(size=(257, 10)) # standard Gaussian
W = preprocessing.MaxAbsScaler().fit_transform(vals) # uniform (on [-1, 1])

delta_w = np.full((257, 10), 0.00)

# encode the desired label to binary colomns
encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
trainout = pd.DataFrame(encoder.fit_transform(train_out).toarray())
testout = pd.DataFrame(encoder.fit_transform(test_out).toarray())

In [34]:
# train the network on train set
mse = 10
while mse > 0.01:
    prediction = []
    for index, x in train_in.iterrows():
        pre = np.full((1, 9), 0.00)
        net = np.dot(x, W)           # linear regression
        out = 1 / (1 + np.exp(-net)) # logistic regression
        digit = np.argmax(out)
        output = np.insert(pre, digit, 1)
        prediction.append(output)
        # generate the delta w using neuron learning rule
        for i in range(trainout.shape[1]): 
            w_tran = 0.1 * x * (trainout.iloc[index].iloc[i] - out[i]) * out[i] * (1 - out[i])
            w_tran = w_tran.values
            w_tran = w_tran.reshape(1,-1).T
            delta_w[:, i] = w_tran[:, 0]
        W += delta_w
    mse = mean_squared_error(trainout.values, prediction)

In [35]:
# calculate the accuracy
traincorrect = 0
for i, x in train_in.iterrows():
    pre = np.full((1, 9), 0.00)
    phi = 1 / (1 + np.exp(-np.dot(x, W)))
    output = np.insert(pre, np.argmax(phi), 1)
    if np.array_equiv(trainout.values[i], output):
        traincorrect += 1
correctr_train = traincorrect / len(train_in) * 100

testcorrect = 0
for j, x in test_in.iterrows():
    pre = np.full((1, 9), 0.00)
    phi = 1 / (1 + np.exp(-np.dot(x, W)))
    output = np.insert(pre, np.argmax(phi), 1)
    if np.array_equiv(testout.values[j], output):
        testcorrect += 1
correctr_test = testcorrect / len(test_in) * 100

print('Train percentage correct: %f' % correctr_train)
print('Test percentage correct: %f' % correctr_test)

Train percentage correct: 97.656708
Test percentage correct: 87.800000
